# By Host Address

In [1]:
import re
import numpy as np
import socket
import time

In [ ]:


N = 2
for i in np.arange(1, N+1):
    #command = read_command_via_port(HOST='testlaptop1')
    command = read_command_via_port(HOST='10.185.98.238')

    
    # Tune the laser
    if 'on' in command:
        print(command)
    
    # Tune the laser with dither off
    elif 'off' in command:
        print(command)
    
    elif command == 'shutdown':
        print('Shutting down the laser')
        break
    elif command == 'error':
        print('error')
        continue


In [71]:
command

(1, 193.5, '50', 200.0)

In [2]:
def read_command_via_port(
    
    HOST = 'testlaptop1',  # IP address (server)
    PORT = 9999,        # Port to listen on (non-privileged ports are > 1023)
    N = 1 # number of connections allowed before closing the port
    ):
    
    dither = 'off' # whether to keep the dither on or off after setting the laser
    
    error_message = ' Command not recognized, try again!'
    ok_message = ' OK'
    
    with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as s:
        s.bind((HOST, PORT))
        s.listen()
        conn, addr = s.accept()
        with conn:
            print('Connected by', addr)
            while True:
                data = conn.recv(1024)
                if data:
                    command = data.decode()
                    
                    print('Customer sent: %s' %command)
                    conn.sendall(data)
                    
                    # command format : '193.5 THz 50 C 200 mA dither off'
                    if ' dither off' in command:
                        laser_parameters = command.replace(' dither off', '')
                        dither = 'off'
                        print('Dither off')
                        break
                    
                    # command format : '193.5 THz 50 C 200 mA dither on'
                    elif ' dither on' in command:
                        laser_parameters = command.replace(' dither on', '')
                        dither = 'on'
                        print('Dither on')
                        break
                        
                    elif 'shutdown' in command :
                        return 'shutdown'
                    
                    else :
                        print('Error: command sent by client not recognized: \'%s\' ' %command )
                        conn.sendall( bytes(error_message, encoding='utf8') )
                        return 'error' 
                        
                # The following makes sure only one instance of data is received per establhished connection
                elif not data:
                    break
    
            try:
                F, T, G = re.findall('[0-9-.]+' , laser_parameters) # extract the laser frequency, bench temperature and gain current
                conn.sendall( bytes(ok_message, encoding='utf8') )

                return dither, float(F), T, float(G) # bench is either 25 or 50, do not let it be a float like 25.0 or 50.0 as it causes problems in other functions
            except:
                print(error_message)
                conn.sendall( bytes(error_message, encoding='utf8') )
                return 'error'
        


In [8]:
data.decode()

''